<a href="https://colab.research.google.com/github/Agudelo18/UDEA-ai4eng-20252---Pruebas-Saber-Pro-Colombia/blob/main/03%20-%20modelo%20con%20AdaBoostClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os


In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.34GB/s]


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# 2 - CARGAR LOS DATOS INICIALES
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 3 - APLICAR CORRESPONDENCIAS Y CONVERSIONES
# Convertir valores ordinales en numéricos
tuition_average_mapping = {
    "No pagó matrícula": 0.0, "Menos de 500 mil": 0.25, "Entre 500 mil y menos de 1 millón": 0.75,
    "Entre 1 millón y menos de 2.5 millones": 1.75, "Entre 2.5 millones y menos de 4 millones": 3.25,
    "Entre 4 millones y menos de 5.5 millones": 4.75, "Entre 5.5 millones y menos de 7 millones": 6.25, "Más de 7 millones": 7.5
}
train_df['E_VALORMATRICULAUNIVERSIDAD'] = train_df['E_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)
test_df['E_VALORMATRICULAUNIVERSIDAD'] = test_df['E_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)

# Codificación de FAMI_ESTRATOVIVIENDA
housing_strata_mapping = {"Estrato 1": 1, "Estrato 2": 2, "Estrato 3": 3, "Estrato 4": 4, "Estrato 5": 5, "Estrato 6": 6, "Sin Estrato": None}
train_df['F_ESTRATOVIVIENDA'] = train_df['F_ESTRATOVIVIENDA'].map(housing_strata_mapping)
test_df['F_ESTRATOVIVIENDA'] = test_df['F_ESTRATOVIVIENDA'].map(housing_strata_mapping)

# Codificación de FAMI_TIENEINTERNET y ESTU_PAGOMATRICULAPROPIO
binary_mappings = {"Si": 1, "No": 0}
train_df['F_TIENEINTERNET'] = train_df['F_TIENEINTERNET'].map(binary_mappings)
test_df['F_TIENEINTERNET'] = test_df['F_TIENEINTERNET'].map(binary_mappings)

train_df['E_PAGOMATRICULAPROPIO'] = train_df['E_PAGOMATRICULAPROPIO'].map(binary_mappings)
test_df['E_PAGOMATRICULAPROPIO'] = test_df['E_PAGOMATRICULAPROPIO'].map(binary_mappings)

# Codificación de la variable de destino
target_mapping = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
train_df['RENDIMIENTO_GLOBAL'] = train_df['RENDIMIENTO_GLOBAL'].map(target_mapping)

# 4 - SEPARAR LAS CARACTERÍSTICAS Y EL OBJETIVO
X_train = train_df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = train_df['RENDIMIENTO_GLOBAL']
X_test = test_df.drop(columns=['ID'])

# 5 - PROCESO PARA PRETRATAMIENTO
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

# 6 - ENTRENAR EL MODELO CON AdaBoost
from sklearn.ensemble import AdaBoostClassifier

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', AdaBoostClassifier(
        n_estimators=200,
        learning_rate=0.8,
        random_state=42
    ))
])

pipeline.fit(X_train, y_train)

# 7 - HACER PREDICCIONES SOBRE test.csv
predictions = pipeline.predict(X_test)

# 8 - CONVERTIR LAS PREDICCIONES EN CLASES ORIGINALES
target_inverse_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
predictions_labels = [target_inverse_mapping[p] for p in predictions]

# 9 - CREAR ARCHIVO submission.csv
submission_df = pd.DataFrame({
    "ID": test_df['ID'],
    "RENDIMIENTO_GLOBAL": predictions_labels
})
submission_df.to_csv("submission.csv", index=False)

print("Dimensiones de submission.csv (filas, columnas): ", submission_df.shape)



Dimensiones de submission.csv (filas, columnas):  (296786, 2)
